# 🧹 Workshop Cleanup Instructions

This notebook provides comprehensive cleanup instructions for all resources created during the Unstructured & Structured RAG Agent Workshop. 

**⚠️ Note**: If you are at an AWS event using vended accounts, you do NOT need to run these steps.  The workshop environment will delete these resources for you.

**⚠️ Important**: Running these cleanup steps will permanently delete all workshop resources and cannot be undone. Make sure you have completed all labs and no longer need the resources before proceeding.

## 📋 Resources to Clean Up

This workshop creates the following AWS resources that incur costs:

### Lab 1 - Unstructured Knowledge Base
- Amazon Bedrock Knowledge Base (Unstructured)
- Amazon S3 bucket with sample data
- IAM roles and policies
- Amazon OpenSearch Serverless collection

### Lab 2 - Structured Knowledge Base  
- Amazon Bedrock Knowledge Base (Structured)
- Amazon Redshift Serverless namespace and workgroup
- Amazon S3 bucket for data staging
- IAM roles and policies for Redshift access

### Lab 3 - AgentCore Deployment
- Amazon Bedrock AgentCore Runtime endpoints
- AgentCore Memory instances
- IAM execution roles
- CloudWatch logs and metrics

## 🚀 Getting Started

Before running cleanup commands, ensure you have:
1. Completed all workshop labs
2. AWS credentials configured with appropriate permissions
3. The same AWS region used during the workshop

## 🔧 Setup and Imports

First, let's import the required libraries and set up our AWS clients.

In [ ]:
import os
import json
import time
import boto3
import logging
from datetime import datetime

# Initialize AWS clients
session = boto3.session.Session()
region = session.region_name

s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
bedrock_agent_client = boto3.client('bedrock-agent')
redshift_client = boto3.client('redshift-serverless')
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm')

print(f"AWS Region: {region}")
print(f"Account ID: {sts_client.get_caller_identity()['Account']}")

In [ ]:
# Load stored variables from workshop labs
try:
    %store -r unstructured_kb_id
    %store -r kb_region
    %store -r data_bucket_name
    print(f"✅ Unstructured KB ID: {unstructured_kb_id}")
    print(f"✅ Data Bucket: {data_bucket_name}")
except NameError:
    print("⚠️ Unstructured KB variables not found - may have been cleaned up already")
    unstructured_kb_id = None
    data_bucket_name = None

try:
    %store -r structured_kb_id
    %store -r structured_kb_region
    print(f"✅ Structured KB ID: {structured_kb_id}")
except NameError:
    print("⚠️ Structured KB variables not found - may have been cleaned up already")
    structured_kb_id = None

# Try to get additional variables from SSM parameters
try:
    unstructured_param = ssm_client.get_parameter(Name='/app/intelligent_rag/agentcore/unstructured_kb_id')
    if not unstructured_kb_id:
        unstructured_kb_id = unstructured_param['Parameter']['Value']
        print(f"✅ Retrieved Unstructured KB ID from SSM: {unstructured_kb_id}")
except:
    pass

try:
    structured_param = ssm_client.get_parameter(Name='/app/intelligent_rag/agentcore/structured_kb_id')
    if not structured_kb_id:
        structured_kb_id = structured_param['Parameter']['Value']
        print(f"✅ Retrieved Structured KB ID from SSM: {structured_kb_id}")
except:
    pass

## 📥 Load Workshop Variables

Load the stored variables from the workshop labs. These contain the resource IDs we need for cleanup.

# 🗑️ Lab 1: Unstructured Knowledge Base Cleanup

This section cleans up resources created in Lab 1 - Unstructured Knowledge Base.

**Resources to be deleted:**
- Unstructured Amazon Bedrock Knowledge Base
- S3 bucket with sample data
- Associated IAM roles and policies
- OpenSearch Serverless collection

In [ ]:
# Import the knowledge base utility
if 'Lab 1' in os.getcwd():
    %cd ..

try:
    from utils.knowledge_base import BedrockKnowledgeBase
    print("✅ Successfully imported BedrockKnowledgeBase utility")
except ImportError as e:
    print(f"⚠️ Could not import BedrockKnowledgeBase utility: {e}")
    print("Make sure you're running this from the correct directory")

### Delete Unstructured Knowledge Base

**⚠️ Warning**: This will permanently delete the unstructured knowledge base and all associated resources.

In [ ]:
def cleanup_unstructured_kb():
    """Clean up the unstructured knowledge base and associated resources"""
    if not unstructured_kb_id:
        print("⚠️ No unstructured knowledge base ID found - skipping cleanup")
        return
    
    try:
        print("=============================== Deleting Unstructured Knowledge Base ==============================")
        
        # Create knowledge base instance for cleanup
        unstructured_knowledge_base = BedrockKnowledgeBase(
            kb_name=f"product-reviews-unstructured-kb",  # Name doesn't matter for deletion
            kb_description="Cleanup instance",
            kb_id=unstructured_kb_id
        )
        
        # Delete the knowledge base with all associated resources
        unstructured_knowledge_base.delete_kb(
            delete_s3_bucket=True, 
            delete_iam_roles_and_policies=True
        )
        
        print("✅ Unstructured Knowledge Base cleanup completed successfully!")
        
        # Clean up SSM parameter
        try:
            ssm_client.delete_parameter(Name='/app/intelligent_rag/agentcore/unstructured_kb_id')
            print("✅ Deleted SSM parameter for unstructured KB ID")
        except:
            print("⚠️ SSM parameter for unstructured KB ID not found or already deleted")
            
    except Exception as e:
        print(f"❌ Error during unstructured KB cleanup: {str(e)}")
        print("You may need to manually delete remaining resources in the AWS console")

# Uncomment the line below to execute the cleanup
cleanup_unstructured_kb()

# 🗑️ Lab 2: Structured Knowledge Base Cleanup

This section cleans up resources created in Lab 2 - Structured Knowledge Base.

**Resources to be deleted:**
- Structured Amazon Bedrock Knowledge Base
- Amazon Redshift Serverless workgroup and namespace
- S3 bucket for data staging
- Associated IAM roles and policies

In [ ]:
# Import the structured knowledge base utility
try:
    from utils.structured_knowledge_base import BedrockStructuredKnowledgeBase
    print("✅ Successfully imported BedrockStructuredKnowledgeBase utility")
except ImportError as e:
    print(f"⚠️ Could not import BedrockStructuredKnowledgeBase utility: {e}")
    print("Make sure you're running this from the correct directory")

### Delete Structured Knowledge Base

**⚠️ Warning**: This will permanently delete the structured knowledge base and all associated resources.

In [ ]:
def cleanup_structured_kb():
    """Clean up the structured knowledge base"""
    if not structured_kb_id:
        print("⚠️ No structured knowledge base ID found - skipping cleanup")
        return
    
    try:
        print("=============================== Deleting Structured Knowledge Base ==============================")
        
        # Create structured knowledge base instance for cleanup
        structured_kb = BedrockStructuredKnowledgeBase(
            kb_name="cleanup-instance",  # Name doesn't matter for deletion
            kb_description="Cleanup instance",
            kb_id=structured_kb_id
        )
        
        # Delete the structured knowledge base
        structured_kb.delete_kb(delete_iam_roles_and_policies=True)
        
        print("✅ Structured Knowledge Base deleted successfully!")
        
        # Clean up SSM parameter
        try:
            ssm_client.delete_parameter(Name='/app/intelligent_rag/agentcore/structured_kb_id')
            print("✅ Deleted SSM parameter for structured KB ID")
        except:
            print("⚠️ SSM parameter for structured KB ID not found or already deleted")
            
    except Exception as e:
        print(f"❌ Error during structured KB cleanup: {str(e)}")
        print("You may need to manually delete remaining resources in the AWS console")

# Uncomment the line below to execute the cleanup
#cleanup_structured_kb()

### Delete Redshift Infrastructure

This comprehensive cleanup function will delete all Redshift-related resources including workgroup, namespace, S3 bucket, and IAM role.

In [ ]:
def cleanup_redshift_environment():
    """
    Delete all Redshift-related resources including workgroup, namespace, S3 bucket, and IAM role.
    This function attempts to clean up resources based on common naming patterns.
    """
    import boto3
    import time
    
    # Initialize clients
    session = boto3.session.Session()
    region = session.region_name
    redshift_client = boto3.client('redshift-serverless', region_name=region)
    iam_client = boto3.client('iam')
    s3 = boto3.resource('s3')
    s3_client = boto3.client('s3')
    
    def wait_for_workgroup_deleted(name, poll_interval=10, max_attempts=60):
        """Wait until workgroup is completely deleted"""
        print(f"  Waiting for workgroup {name} to be deleted...")
        attempts = 0
        while attempts < max_attempts:
            try:
                wg = redshift_client.get_workgroup(workgroupName=name)["workgroup"]
                status = wg["status"]
                print(f"    Workgroup status: {status}")
                if status == "DELETED":
                    break
                time.sleep(poll_interval)
                attempts += 1
            except redshift_client.exceptions.ResourceNotFoundException:
                print("    Workgroup deleted successfully")
                return
        
        if attempts >= max_attempts:
            print(f"    Warning: Timeout waiting for workgroup deletion after {max_attempts * poll_interval} seconds")
    
    def wait_for_namespace_deleted(name, poll_interval=10, max_attempts=60):
        """Wait until namespace is completely deleted"""
        print(f"  Waiting for namespace {name} to be deleted...")
        attempts = 0
        while attempts < max_attempts:
            try:
                redshift_client.get_namespace(namespaceName=name)
                print(f"    Namespace still exists, waiting...")
                time.sleep(poll_interval)
                attempts += 1
            except redshift_client.exceptions.ResourceNotFoundException:
                print("    Namespace deleted successfully")
                return
        
        if attempts >= max_attempts:
            print(f"    Warning: Timeout waiting for namespace deletion after {max_attempts * poll_interval} seconds")
    
    print("Starting Redshift environment cleanup...")
    print("=" * 60)
    
    # Find Redshift resources by listing them
    try:
        workgroups = redshift_client.list_workgroups()['workgroups']
        namespaces = redshift_client.list_namespaces()['namespaces']
        
        # Look for workshop-related resources
        workshop_workgroups = [wg for wg in workgroups if 'sds-ecommerce' in wg['workgroupName']]
        workshop_namespaces = [ns for ns in namespaces if 'sds-ecommerce' in ns['namespaceName']]
        
        # Delete workgroups
        for workgroup in workshop_workgroups:
            wg_name = workgroup['workgroupName']
            print(f"Step 1: Deleting Redshift workgroup {wg_name}")
            try:
                redshift_client.delete_workgroup(workgroupName=wg_name)
                print("  Workgroup deletion initiated")
                wait_for_workgroup_deleted(wg_name)
            except Exception as e:
                print(f"  Error deleting workgroup: {str(e)}")
        
        # Delete namespaces
        for namespace in workshop_namespaces:
            ns_name = namespace['namespaceName']
            print(f"\nStep 2: Deleting Redshift namespace {ns_name}")
            try:
                redshift_client.delete_namespace(namespaceName=ns_name)
                print("  Namespace deletion initiated")
                wait_for_namespace_deleted(ns_name)
            except Exception as e:
                print(f"  Error deleting namespace: {str(e)}")
        
        if not workshop_workgroups and not workshop_namespaces:
            print("No workshop-related Redshift resources found")
            
    except Exception as e:
        print(f"Error listing Redshift resources: {str(e)}")
    
    # Clean up S3 buckets with workshop naming pattern
    print(f"\nStep 3: Cleaning up workshop S3 buckets")
    try:
        buckets = s3_client.list_buckets()['Buckets']
        workshop_buckets = [b for b in buckets if 'sds-ecommerce' in b['Name'] or 'structured-kb' in b['Name']]
        
        for bucket_info in workshop_buckets:
            bucket_name = bucket_info['Name']
            try:
                bucket = s3.Bucket(bucket_name)
                
                # Delete all objects in the bucket
                print(f"  Emptying bucket {bucket_name}...")
                objects_to_delete = []
                for obj in bucket.objects.all():
                    objects_to_delete.append({'Key': obj.key})
                
                if objects_to_delete:
                    bucket.delete_objects(Delete={'Objects': objects_to_delete})
                    print(f"    Deleted {len(objects_to_delete)} objects")
                
                # Delete the bucket
                print(f"  Deleting bucket {bucket_name}...")
                bucket.delete()
                print(f"  S3 bucket {bucket_name} deleted successfully")
                
            except Exception as e:
                print(f"  Error deleting S3 bucket {bucket_name}: {str(e)}")
                
    except Exception as e:
        print(f"Error cleaning up S3 buckets: {str(e)}")
    
    # Clean up IAM roles with workshop naming pattern
    print(f"\nStep 4: Cleaning up workshop IAM roles")
    try:
        roles = iam_client.list_roles()['Roles']
        workshop_roles = [r for r in roles if 'RedshiftRole' in r['RoleName'] or 'sds-ecommerce' in r['RoleName']]
        
        for role in workshop_roles:
            role_name = role['RoleName']
            try:
                # Detach managed policies
                print(f"  Cleaning up role {role_name}...")
                attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
                for policy in attached_policies:
                    policy_arn = policy['PolicyArn']
                    iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
                    print(f"    Detached policy: {policy['PolicyName']}")
                    
                    # Delete custom policies (not AWS managed)
                    if not policy_arn.startswith('arn:aws:iam::aws:policy/'):
                        try:
                            iam_client.delete_policy(PolicyArn=policy_arn)
                            print(f"    Deleted custom policy: {policy['PolicyName']}")
                        except Exception as e:
                            print(f"    Could not delete policy {policy['PolicyName']}: {str(e)}")
                
                # Delete inline policies
                inline_policies = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
                for policy_name in inline_policies:
                    iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
                    print(f"    Deleted inline policy: {policy_name}")
                
                # Delete the role
                iam_client.delete_role(RoleName=role_name)
                print(f"  IAM role {role_name} deleted successfully")
                
            except Exception as e:
                print(f"  Error deleting IAM role {role_name}: {str(e)}")
                
    except Exception as e:
        print(f"Error cleaning up IAM roles: {str(e)}")
    
    print("\n" + "=" * 60)
    print("Redshift environment cleanup completed")

# Uncomment the line below to execute the Redshift cleanup
#cleanup_redshift_environment()

# 🗑️ Lab 3: AgentCore Deployment Cleanup

This section cleans up resources created in Lab 3 - AgentCore Deployment and Memory.

**Resources to be deleted:**
- Amazon Bedrock AgentCore Runtime endpoints
- AgentCore Memory instances
- IAM execution roles
- CloudWatch logs and metrics

In [ ]:
def cleanup_agentcore_resources():
    """Clean up AgentCore runtime endpoints and memory instances"""
    try:
        # Import AgentCore toolkit
        from bedrock_agentcore_starter_toolkit import Runtime
        print("✅ Successfully imported AgentCore toolkit")
        
        # Initialize AgentCore client
        agentcore_client = boto3.client('bedrock-agentcore-control')
        
        print("=============================== Cleaning up AgentCore Resources ==============================")
        
        # List all runtime endpoints
        try:
            runtimes = agentcore_client.list_runtime_endpoints()['runtimeEndpoints']
            workshop_runtimes = [r for r in runtimes if 'intelligent_rag_agent' in r.get('name', '')]
            
            for runtime in workshop_runtimes:
                runtime_name = runtime['name']
                runtime_arn = runtime['arn']
                
                print(f"Deleting AgentCore runtime: {runtime_name}")
                try:
                    agentcore_client.delete_runtime_endpoint(runtimeEndpointArn=runtime_arn)
                    print(f"✅ Deleted runtime endpoint: {runtime_name}")
                except Exception as e:
                    print(f"❌ Error deleting runtime {runtime_name}: {str(e)}")
                    
            if not workshop_runtimes:
                print("⚠️ No workshop-related AgentCore runtimes found")
                
        except Exception as e:
            print(f"❌ Error listing AgentCore runtimes: {str(e)}")
        
        # Clean up AgentCore Memory instances
        try:
            agentcore_memory_client = boto3.client('bedrock-agentcore-memory')
            memories = agentcore_memory_client.list_memories()['memories']
            
            for memory in memories:
                memory_id = memory['memoryId']
                memory_name = memory.get('name', 'Unknown')
                
                print(f"Deleting AgentCore memory: {memory_name} ({memory_id})")
                try:
                    agentcore_memory_client.delete_memory(memoryId=memory_id)
                    print(f"✅ Deleted memory: {memory_name}")
                except Exception as e:
                    print(f"❌ Error deleting memory {memory_name}: {str(e)}")
                    
        except Exception as e:
            print(f"⚠️ Could not clean up AgentCore memories: {str(e)}")
        
        # Clean up SSM parameters
        try:
            ssm_client.delete_parameter(Name='/app/intelligent_rag/agentcore/memory_id')
            print("✅ Deleted SSM parameter for memory ID")
        except:
            print("⚠️ SSM parameter for memory ID not found or already deleted")
        
        print("✅ AgentCore resources cleanup completed!")
        
    except ImportError:
        print("⚠️ AgentCore toolkit not available - skipping AgentCore cleanup")
        print("You may need to manually delete AgentCore resources in the AWS console")
    except Exception as e:
        print(f"❌ Error during AgentCore cleanup: {str(e)}")
        print("You may need to manually delete remaining resources in the AWS console")

# Uncomment the line below to execute the AgentCore cleanup
#cleanup_agentcore_resources()

### Clean up IAM Roles for AgentCore

Clean up IAM roles created for AgentCore execution.

In [ ]:
def cleanup_agentcore_iam_roles():
    """Clean up IAM roles created for AgentCore"""
    try:
        print("=============================== Cleaning up AgentCore IAM Roles ==============================")
        
        # List all roles and find AgentCore-related ones
        roles = iam_client.list_roles()['Roles']
        agentcore_roles = [r for r in roles if 'AgentCoreExecutionRole' in r['RoleName'] or 'intelligent-rag-agent' in r['RoleName']]
        
        for role in agentcore_roles:
            role_name = role['RoleName']
            try:
                print(f"Cleaning up IAM role: {role_name}")
                
                # Detach managed policies
                attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
                for policy in attached_policies:
                    policy_arn = policy['PolicyArn']
                    iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
                    print(f"  Detached policy: {policy['PolicyName']}")
                    
                    # Delete custom policies (not AWS managed)
                    if not policy_arn.startswith('arn:aws:iam::aws:policy/'):
                        try:
                            iam_client.delete_policy(PolicyArn=policy_arn)
                            print(f"  Deleted custom policy: {policy['PolicyName']}")
                        except Exception as e:
                            print(f"  Could not delete policy {policy['PolicyName']}: {str(e)}")
                
                # Delete inline policies
                inline_policies = iam_client.list_role_policies(RoleName=role_name)['PolicyNames']
                for policy_name in inline_policies:
                    iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
                    print(f"  Deleted inline policy: {policy_name}")
                
                # Delete the role
                iam_client.delete_role(RoleName=role_name)
                print(f"✅ Deleted IAM role: {role_name}")
                
            except Exception as e:
                print(f"❌ Error deleting IAM role {role_name}: {str(e)}")
        
        if not agentcore_roles:
            print("⚠️ No AgentCore-related IAM roles found")
        
        print("✅ AgentCore IAM roles cleanup completed!")
        
    except Exception as e:
        print(f"❌ Error during AgentCore IAM cleanup: {str(e)}")

# Uncomment the line below to execute the AgentCore IAM cleanup
#cleanup_agentcore_iam_roles()

# 🚀 Complete Workshop Cleanup

This section provides a comprehensive cleanup that runs all the cleanup functions in the correct order.

**⚠️ Final Warning**: This will delete ALL workshop resources permanently. Make sure you have completed all labs and no longer need any of the resources.

In [ ]:
def complete_workshop_cleanup():
    """Run complete cleanup of all workshop resources"""
    print("\n" + "=" * 80)
    print("                    COMPLETE WORKSHOP CLEANUP")
    print("=" * 80)
    print("\n⚠️  WARNING: This will permanently delete ALL workshop resources!")
    print("\nStarting comprehensive cleanup in 5 seconds...")
    
    import time
    for i in range(5, 0, -1):
        print(f"  {i}...")
        time.sleep(1)
    
    print("\n🚀 Starting cleanup process...\n")
    
    # Step 1: Clean up AgentCore resources first (they depend on other resources)
    print("Step 1: Cleaning up AgentCore resources...")
    cleanup_agentcore_resources()
    cleanup_agentcore_iam_roles()
    
    # Step 2: Clean up Knowledge Bases
    print("\nStep 2: Cleaning up Knowledge Bases...")
    cleanup_unstructured_kb()
    cleanup_structured_kb()
    
    # Step 3: Clean up Redshift infrastructure
    print("\nStep 3: Cleaning up Redshift infrastructure...")
    cleanup_redshift_environment()
    
    # Step 4: Clean up any remaining SSM parameters
    print("\nStep 4: Cleaning up SSM parameters...")
    ssm_parameters = [
        '/app/intelligent_rag/agentcore/unstructured_kb_id',
        '/app/intelligent_rag/agentcore/structured_kb_id',
        '/app/intelligent_rag/agentcore/memory_id'
    ]
    
    for param_name in ssm_parameters:
        try:
            ssm_client.delete_parameter(Name=param_name)
            print(f"✅ Deleted SSM parameter: {param_name}")
        except:
            print(f"⚠️ SSM parameter not found or already deleted: {param_name}")
    
    print("\n" + "=" * 80)
    print("                    CLEANUP COMPLETED")
    print("=" * 80)
    print("\n✅ Workshop cleanup completed successfully!")
    print("\n💰 All AWS resources have been deleted to avoid ongoing charges.")
    print("\n📝 Summary of cleaned up resources:")
    print("   • Amazon Bedrock Knowledge Bases (Unstructured & Structured)")
    print("   • Amazon S3 buckets and data")
    print("   • Amazon Redshift Serverless workgroups and namespaces")
    print("   • Amazon Bedrock AgentCore runtime endpoints")
    print("   • AgentCore Memory instances")
    print("   • IAM roles and policies")
    print("   • SSM parameters")
    print("   • CloudWatch logs and metrics (will expire automatically)")
    
    print("\n🎉 Thank you for completing the Unstructured & Structured RAG Agent Workshop!")

# UNCOMMENT THE LINE BELOW TO RUN COMPLETE CLEANUP
complete_workshop_cleanup()

## 📊 Cost Monitoring

After cleanup, monitor your AWS costs to ensure all resources have been properly deleted:

1. **AWS Cost Explorer**: Check for any ongoing charges related to:
   - Amazon Bedrock
   - Amazon S3
   - Amazon Redshift
   - Amazon OpenSearch
   - CloudWatch

2. **AWS Billing Dashboard**: Review your current month's charges

3. **CloudWatch Logs**: Log groups will expire automatically based on retention settings

## 🎆 Workshop Complete!

Congratulations on completing the Unstructured & Structured RAG Agent Workshop! 

You've successfully:
- ✅ Built an unstructured knowledge base with Amazon Bedrock
- ✅ Created a structured knowledge base with Amazon Redshift
- ✅ Deployed intelligent RAG agents to AgentCore
- ✅ Implemented memory and observability features
- ✅ Cleaned up all AWS resources

### Next Steps
- Explore more [Amazon Bedrock workshops](https://github.com/aws-samples/amazon-bedrock-workshop)
- Learn about [AgentCore best practices](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore.html)
- Build your own RAG applications using the patterns from this workshop

**Happy building! 🚀**

## 🔧 Manual Cleanup Instructions

If the automated cleanup fails or you prefer to clean up resources manually, follow these steps in the AWS Console:

### 1. Amazon Bedrock Knowledge Bases
1. Go to [Amazon Bedrock Console](https://console.aws.amazon.com/bedrock/)
2. Navigate to "Knowledge bases" in the left sidebar
3. Delete any knowledge bases with names containing:
   - `product-reviews-unstructured-kb`
   - `redshift-structured-kb`

### 2. Amazon Redshift Serverless
1. Go to [Amazon Redshift Console](https://console.aws.amazon.com/redshiftv2/)
2. Navigate to "Serverless dashboard"
3. Delete workgroups with names containing `sds-ecommerce`
4. Delete namespaces with names containing `sds-ecommerce`

### 3. Amazon S3 Buckets
1. Go to [Amazon S3 Console](https://console.aws.amazon.com/s3/)
2. Delete buckets with names containing:
   - `product-reviews-unstructured`
   - `sds-ecommerce`
   - `structured-kb`

### 4. Amazon Bedrock AgentCore
1. Go to [Amazon Bedrock Console](https://console.aws.amazon.com/bedrock/)
2. Navigate to "AgentCore" section
3. Delete runtime endpoints with names containing `intelligent_rag_agent`
4. Delete memory instances created during the workshop

### 5. IAM Roles and Policies
1. Go to [IAM Console](https://console.aws.amazon.com/iam/)
2. Navigate to "Roles"
3. Delete roles with names containing:
   - `BedrockKnowledgeBaseRole`
   - `RedshiftRole`
   - `AgentCoreExecutionRole`
   - `intelligent-rag-agent`

### 6. Systems Manager Parameters
1. Go to [Systems Manager Console](https://console.aws.amazon.com/systems-manager/)
2. Navigate to "Parameter Store"
3. Delete parameters under `/app/intelligent_rag/agentcore/`